<a href="https://colab.research.google.com/github/Kussil/Financial_Sentiment_LLM/blob/main/03_Sentiment_Analysis/Revised_notebooks/Complete_Colab_LLama_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Sentiment Analysis for Oil & Gas Aricles using Higging Face Llama

## Overview
placeholder

# Dependencies and Libraries installation part

In [1]:
# Installing necessary dependencies for the Llama worflow
!pip install -q -U langchain langchain_community transformers bitsandbytes accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.0/990.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 48.9 MB/s eta 0:00:00


In [2]:
# import libraries
import os
import json
import pandas as pd
import torch
from tqdm import tqdm
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline
from huggingface_hub import HfFolder, HfApi
from google.colab import userdata
import re
import time

# Connecting to G-Drive, GitHub and Hugging Face API

In [3]:
# connecting to the user's google drive to save output files computed by current notebook
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Hugging Face connection using API Key from Secret Keys

hf_token = userdata.get('HF_TOKEN')
if hf_token:
    HfFolder.save_token(hf_token)
    api = HfApi()
    user_info = api.whoami()
    if user_info:
        print("Connection to Hugging Face was successful.")
    else:
        print("Failed to connect to Hugging Face. Please check your token.")
else:
    print("Hugging Face token not found. Please set the HF_TOKEN environment variable.")


Connection to Hugging Face was successful.


In [5]:
# Connection to GitHub repository
# Import GitHub token with Google secrets and clone the repository
GITHUB_TOKEN = userdata.get('github')
os.environ['GITHUB_TOKEN'] = GITHUB_TOKEN
!git clone  https://{GITHUB_TOKEN}@github.com/Kussil/Financial_Sentiment_LLM.git

Cloning into 'Financial_Sentiment_LLM'...
remote: Enumerating objects: 2925, done.
remote: Counting objects: 100% (491/491), done.
remote: Compressing objects: 100% (256/256), done.
remote: Total 2925 (delta 280), reused 409 (delta 231), pack-reused 2434
Receiving objects: 100% (2925/2925), 465.26 MiB | 14.72 MiB/s, done.
Resolving deltas: 100% (1912/1912), done.
Updating files: 100% (1207/1207), done.


# LLAMA PART

In [6]:
import sys
sys.path.append('/content/Financial_Sentiment_LLM/10_Source_Code/')

In [7]:
# Import the necessary modules from repository
import data_setup as ds


In [8]:
# Define the template
TEMPLATE = """<s>Classify the following article into categories with sentiment (Positive, Neutral, Negative, N/A if not applicable or not mentioned) and provide the output in the specified dictionary format.
Example:
Article: ExxonMobil announced a significant increase in quarterly profits due to rising oil prices and increased production levels.
Output: {{"Finance": "Positive", 'Production': "Positive", "Reserves / Exploration / Acquisitions / Mergers / Divestments": 'Neutral', "Environment / Regulatory / Geopolitics": 'Neutral', "Alternative Energy / Lower Carbon": 'Neutral', "Oil Price / Natural Gas Price / Gasoline Price": "Positive"}}

Example:
Article: Chevron plans to invest heavily in renewable energy projects, aiming to reduce its carbon footprint over the next decade.
Output: {{'Finance': 'Neutral', 'Production': 'Neutral', "Reserves / Exploration / Acquisitions / Mergers / Divestments": 'Neutral', "Environment / Regulatory / Geopolitics": "Positive", "Alternative Energy / Lower Carbon": "Positive", "Oil Price / Natural Gas Price / Gasoline Price": 'Neutral'}}

Example:
Article: BP faced regulatory challenges in its latest drilling project, delaying operations and increasing costs.
Output: {{'Finance': 'Negative', "Production": 'Negative', "Reserves / Exploration / Acquisitions / Mergers / Divestments": 'Negative', "Environment / Regulatory / Geopolitics": 'Negative', "Alternative Energy / Lower Carbon": 'Neutral', "Oil Price / Natural Gas Price / Gasoline Price": 'Neutral'}}

Article: {article}

Output only the EXACT dictionary format:
{{"Finance": '[Sentiment]', "Production": '[Sentiment]', "Reserves / Exploration / Acquisitions / Mergers / Divestments": '[Sentiment]', "Environment / Regulatory / Geopolitics":: '[Sentiment]', "Alternative Energy / Lower Carbon": '[Sentiment]', "Oil Price / Natural Gas Price / Gasoline Price": '[Sentiment]'}}

Do not use any other format or additional information. Please provide the output in the specified format only.</s>"""

In [16]:
# Global Variables
CATEGORIES = [
        "Finance",
        "Production",
        "Reserves / Exploration / Acquisitions / Mergers / Divestments",
        "Environment / Regulatory / Geopolitics",
        "Alternative Energy / Lower Carbon",
        "Oil Price / Natural Gas Price / Gasoline Price"
        ]

# Define the file path to save in Google Drive
DRIVE_PATH = '/content/drive/MyDrive'
SENTIMENT_RESULTS_FILE_PATH = os.path.join(DRIVE_PATH, 'Full_data_LLama_model_sentiment_analysis_results.csv')
ROWS_TO_DROP = ['PQ-2840736837']

MAX_TRIES = 5

In [17]:
# Load and prepare data
text_df = ds.load_cleaned_data()
text_df = ds.drop_unprocessable_rows(text_df, ROWS_TO_DROP)

In [18]:
# Check if sentiment analysis results file exists and create if it doesn't
if not ds.check_file_exists(SENTIMENT_RESULTS_FILE_PATH):
    empty_sentiment_df = ds.create_empty_sentiment_df(text_df, CATEGORIES)
    ds.save_dataframe_to_csv(empty_sentiment_df, SENTIMENT_RESULTS_FILE_PATH)
    print(f"Created and saved an empty sentiment analysis DataFrame to {SENTIMENT_RESULTS_FILE_PATH}")
else:
    print(f"The file exists in the current directory.")

Created and saved an empty sentiment analysis DataFrame to /content/drive/MyDrive/Full_data_LLama_model_sentiment_analysis_results.csv


In [19]:
# Load the LLaMA model and tokenizer
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    device_map="auto",
    quantization_config=quantization_config,
    use_auth_token=hf_token
)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct", use_auth_token=hf_token)

# Create text generation pipeline
generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    use_cache=True,
    device_map="auto",
    max_length=8192,  # increase to max token size supported by the model
    truncation=True,
    do_sample=True,
    top_k=5,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:469: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:778: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [20]:
text_df.head()

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN
1,Investment Research,IR-2,EOG,2024-05-14,"EOG Resources, Inc.","Stock Report | May 14, 2024 | NYSESymbol: EOG ...",NaN
2,Investment Research,IR-3,EOG,2024-05-11,"EOG Resources, Inc.","Stock Report | May 11, 2024 | NYSESymbol: EOG ...",NaN
3,Investment Research,IR-4,DVN,2024-05-11,Devon Energy Corporation,"Stock Report | May 11, 2024 | NYSESymbol: DVN ...",NaN
4,Investment Research,IR-5,COP,2024-05-07,ConocoPhillips,"Stock Report | May 07, 2024 | NYSESymbol:\n\nC...",NaN


In [22]:
# Use the already existing Hugging Face pipeline instead of ollama
def get_hf_llama_response(article, template, pipeline):
    prompt = template.format(article=article)
    response = pipeline(prompt)
    if response:
        return response[0]['generated_text'].strip()
    else:
        return "Error: No response generated."

In [24]:
# Function to process a single unique ID using Hugging Face's pipeline
def process_unique_id(unique_id, pipeline):
    for _ in range(MAX_TRIES):
        try:
            company, source, headline, text = ds.get_model_inputs(text_df, unique_id)
            response = get_hf_llama_response(text, TEMPLATE, pipeline)
            sentiment_dict = ds.extract_and_convert_to_dict(response)

            if isinstance(sentiment_dict, dict):
                ds.update_csv(SENTIMENT_RESULTS_FILE_PATH, unique_id, sentiment_dict, CATEGORIES)
                return True
            print("Error: Sentiment dictionary not found. Retrying...")
        except Exception as e:
            print(f"Error: {e}. Retrying...")

    print(f"Max retries reached for Unique_ID '{unique_id}'. Inserting 'No JSON found' for each category.")
    sentiment_dict = {category: "No JSON found" for category in CATEGORIES}
    ds.update_csv(SENTIMENT_RESULTS_FILE_PATH, unique_id, sentiment_dict, CATEGORIES)
    return False

In [ ]:
# Main processing loop using the Hugging Face pipeline
start_time = time.time()
count = 0

pipeline = generation_pipeline  # Assuming 'generation_pipeline' is set up from the previous script

while True:
    unique_id = ds.find_first_unique_id_with_empty_values(SENTIMENT_RESULTS_FILE_PATH, CATEGORIES)
    if not unique_id:
        break

    if process_unique_id(unique_id, pipeline):
        count += 1
        if count % 10 == 0:
            elapsed_time = time.time() - start_time
            minutes, seconds = divmod(elapsed_time, 60)
            print(f"Iteration: {count}, Elapsed Time: {int(minutes)} minutes and {seconds:.2f} seconds")

print("Processing complete.")


Row with Unique_ID 'IR-1' has been updated.
